# NLP Fuzzy matching protype code

In [117]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import string
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [3]:
spark = SparkSession.builder.appName("My NLP app").getOrCreate()

21/12/24 14:55:33 WARN Utils: Your hostname, maxs-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.1.217 instead (on interface en0)
21/12/24 14:55:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/24 14:55:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sql("show tables in default").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [6]:
cols = ["product Family", "Product", "Platform"]

sdf = (spark.read.format("csv").option("header", True).load("/Users/maxnethercott/Documents/data_science_projects/nlp_sentence_matching/fuzzy_prototype/")
        .select([F.col(x).alias(x.lower().strip()) for x in cols]))


sdf.distinct().limit(10).show(1000,False)


21/12/24 14:56:35 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , , 
 Schema: Product Family, Product, Platform
Expected: Product Family but found: 
CSV file: file:///Users/maxnethercott/Documents/data_science_projects/nlp_sentence_matching/fuzzy_prototype/README.md
21/12/24 14:56:35 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , , 
 Schema: Product Family, Product, Platform
Expected: Product Family but found: 
CSV file: file:///Users/maxnethercott/Documents/data_science_projects/nlp_sentence_matching/fuzzy_prototype/nlp_fuzzy_matcher.ipynb


+------------------+----------------------------------------------------------------+-----------------------------------------------+
|product family    |product                                                         |platform                                       |
+------------------+----------------------------------------------------------------+-----------------------------------------------+
|Microsoft Office  |Microsoft SharePoint Server 2019                                |null                                           |
|Developer Tools   |Microsoft Visual Studio 2017 version 15.9 (includes 15.0 - 15.8)|null                                           |
|Browser           |Microsoft Edge (Chromium-based) in IE Mode                      |Windows 10 Version 1909 for ARM64-based Systems|
|Browser           |Microsoft Edge (Chromium-based) in IE Mode                      |Windows 11 for ARM64-based Systems             |
|Developer Tools   |Microsoft Visual Studio 2015 Update 3     

In [59]:
# Columns we are interested in
cols = ["Product", "Platform"]

#Read data
pdf = pd.read_csv("/Users/maxnethercott/Documents/data_science_projects/nlp_sentence_matching/fuzzy_prototype/ms_security_updates_csv.csv")
pdf = pdf[cols]
pdf.columns = pdf.columns.str.lower()
pdf = pdf.fillna("")

#Lower all pandas column values
pdf = pdf.apply(lambda x: x.astype(str).str.lower())

pdf['document'] = pdf[pdf.columns].agg('. '.join, axis=1)
pdf.drop_duplicates(inplace=True)


In [102]:
doc_list = pdf["document"].tolist()
tokenized_sents = [word_tokenize(i) for i in doc_list]
tokenized_sents

[['microsoft', 'sharepoint', 'server', '2019', '.'],
 ['microsoft',
  'sharepoint',
  'enterprise',
  'server',
  '2013',
  'service',
  'pack',
  '1',
  '.'],
 ['microsoft', 'sharepoint', 'enterprise', 'server', '2016', '.'],
 ['microsoft', 'edge', '(', 'chromium-based', ')', '.'],
 ['microsoft', 'biztalk', 'esb', 'toolkit', '2.3', '.'],
 ['microsoft', 'biztalk', 'esb', 'toolkit', '2.4', '.'],
 ['microsoft', 'biztalk', 'esb', 'toolkit', '2.2', '.'],
 ['visual', 'studio', 'code', '.'],
 ['office', 'app', '.'],
 ['microsoft', '4k', 'wireless', 'display', 'adapter', '.'],
 ['powershell', '7.2', '.'],
 ['windows',
  'server',
  '2012',
  'r2',
  '(',
  'server',
  'core',
  'installation',
  ')',
  '.'],
 ['windows', 'server', '2012', 'r2', '.'],
 ['windows',
  'server',
  '2012',
  '(',
  'server',
  'core',
  'installation',
  ')',
  '.'],
 ['windows', 'server', '2012', '.'],
 ['windows',
  'server',
  '2008',
  'r2',
  'for',
  'x64-based',
  'systems',
  'service',
  'pack',
  '1',
  

In [114]:
#Filter out stopwords...
stop_words = set(nltk.corpus.stopwords.words("english"))

out = [[w for w in x if not w in list(stop_words)] for x in tokenized_sents]

nested_l = []
for _ in out:
    nested_l.append([s.translate(str.maketrans("", "", string.punctuation)) for s in _])

detokenized_sents = [TreebankWordDetokenizer().detokenize(i) for i in nested_l]
detokenized_sents = list(set(detokenized_sents))


In [115]:
detokenized_sents

['microsoft dynamics 365  onpremises  version 91',
 'remote desktop client windows desktop',
 'windows rt 81',
 'microsoft excel 2013 service pack 1  64bit editions',
 'microsoft defender iot',
 'microsoft office 2019 64bit editions',
 'windows 10 x64based systems',
 'vp9 video extensions',
 'microsoft edge  chromiumbased  ie mode  windows 10 version 2004 32bit systems',
 'microsoft edge  chromiumbased  ie mode  windows 11 x64based systems',
 'system center 2012 r2 operations manager',
 'aspnet core 60',
 'windows 10 version 2004 32bit systems',
 'azure site recovery',
 'microsoft edge  chromiumbased  ie mode  windows 10 version 2004 x64based systems',
 'microsoft edge  chromiumbased  ie mode  windows 10 version 1909 32bit systems',
 'windows server  version 20h2  server core installation',
 '3d viewer',
 'microsoft excel 2013 rt service pack 1',
 'microsoft edge  chromiumbased  ie mode  windows 10 version 20h2 x64based systems',
 'windows 7 32bit systems service pack 1',
 'azure activ